In [1]:
##installs pytorch on a cuda-capable windows machine using pip

!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

!pip install torchsummary

!pip install numpy

!pip install matplotlib

!pip install opencv-python

!pip install scikit-learn

!pip install pandas

Looking in indexes: https://download.pytorch.org/whl/cu118


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import torch
import torch.nn as nn
import os
import cv2
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [3]:
from torchvision import models
from torchvision import transforms
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [4]:
os.chdir("..")
os.path.split(os.getcwd())[1]

'progettoVIPM'

In [5]:

# images_names = []
# labels = []
# with open('train_small.csv', newline='') as csvfile:
#     reader = csv.reader(csvfile)
#     for row in reader:
#         images_names.append("./train_set/"+row[0])
#         labels.append(row[1])
# labels = np.array(labels)
# print(images_names[0:5])
# print(labels[0:5])
# print(np.iinfo(cv2.imread(images_names[0]).dtype))
# images = []
# # Massimi valori ottenuti nel dataset di training labelled
# # max_righe = 1408 
# # max_colonne = 1404
# # max_righe = 0
# # max_colonne = 0
# for name in images_names:
#     try:
#         im = cv2.resize((cv2.imread(name, cv2.IMREAD_COLOR).astype(np.double)/255), (227,227), interpolation=cv2.INTER_CUBIC)
#         # print(im.shape)
#         images.append(im)
#         # if(im.shape[0] > max_righe):
#         #     max_righe = im.shape[0]
#         # if(im.shape[1] > max_colonne):
#         #     max_colonne = im.shape[1]
#     except:
#         print("Errore in immagine: {}".format(name))
# # print(max_righe)
# # print(max_colonne)
# images = np.array(images)
# print(images.shape)
# print(labels.shape)
# concat = np.concatenate((images, labels), axis=0)
# print(concat.shape)


In [6]:
from torch.utils.data import Dataset
from enum import Enum

class datasets(Enum):
    TRAINING_LABELED = ["train_small.csv", "train_set"]
    TRAINING_UNLABELED = ["train_unlabeled.csv", "train_set"]
    TEST = ["val_info.csv", "val_set"]
    TEST_DEGRADED = ["val_info.csv", "val_set_degraded"]

class networks(Enum):
    ALEXNET = 277
    RESNET50 = 224
    GOOGLENET = 224
    DEFAULT = 224

class ImageDataset(Dataset):
    def __init__(self, dataset, network_input_size):
        super().__init__()
        self.images_names = []
        self.labels = []
        dataset = dataset.value
        annotations_file = dataset[0]
        img_dir = dataset[1]
        with open(annotations_file, newline='') as csvfile:
            reader = csv.reader(csvfile)
            for row in reader:
                self.images_names.append("./{}/{}".format(img_dir, row[0]))
                self.labels.append(row[1])
        self.images_names = np.array(self.images_names)
        self.labels = np.array(self.labels)
        # in base al valore passato si sceglie la rete che utilizzerà il dataset, serve per modificare le dimensioni delle immagini
        self.im_size = network_input_size.value
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, index):
        #moveaxis serve per avere come dimensione dell'immagine (3, righe, colonne) invece di (righe, colonne, 3)
        image = np.moveaxis(cv2.resize((cv2.imread(self.images_names[index], cv2.IMREAD_COLOR).astype(np.double)/255), 
                                       (self.im_size,self.im_size), 
                                        interpolation=cv2.INTER_CUBIC).astype(np.float32),
                            -1, 0)
        # eventualmente si può aggiungere l'alternativa di fare random cropping dell'immagine
        label = self.labels[index]
        return torch.from_numpy(image).cuda(), label

In [7]:
from torchsummary import summary
dataset_train_AlexNet = ImageDataset(dataset=datasets.TRAINING_LABELED, network_input_size=networks.ALEXNET)
alexnet = models.alexnet(pretrained=True)
alexnet.cuda()
summary(alexnet, (3, 224, 224))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 55, 55]          23,296
              ReLU-2           [-1, 64, 55, 55]               0
         MaxPool2d-3           [-1, 64, 27, 27]               0
            Conv2d-4          [-1, 192, 27, 27]         307,392
              ReLU-5          [-1, 192, 27, 27]               0
         MaxPool2d-6          [-1, 192, 13, 13]               0
            Conv2d-7          [-1, 384, 13, 13]         663,936
              ReLU-8          [-1, 384, 13, 13]               0
            Conv2d-9          [-1, 256, 13, 13]         884,992
             ReLU-10          [-1, 256, 13, 13]               0
           Conv2d-11          [-1, 256, 13, 13]         590,080
             ReLU-12          [-1, 256, 13, 13]               0
        MaxPool2d-13            [-1, 256, 6, 6]               0
AdaptiveAvgPool2d-14            [-1, 25

In [8]:
alexnet.classifier = nn.Sequential(*[alexnet.classifier[i] for i in range(5)])
alexnet.cuda()
summary(alexnet, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 55, 55]          23,296
              ReLU-2           [-1, 64, 55, 55]               0
         MaxPool2d-3           [-1, 64, 27, 27]               0
            Conv2d-4          [-1, 192, 27, 27]         307,392
              ReLU-5          [-1, 192, 27, 27]               0
         MaxPool2d-6          [-1, 192, 13, 13]               0
            Conv2d-7          [-1, 384, 13, 13]         663,936
              ReLU-8          [-1, 384, 13, 13]               0
            Conv2d-9          [-1, 256, 13, 13]         884,992
             ReLU-10          [-1, 256, 13, 13]               0
           Conv2d-11          [-1, 256, 13, 13]         590,080
             ReLU-12          [-1, 256, 13, 13]               0
        MaxPool2d-13            [-1, 256, 6, 6]               0
AdaptiveAvgPool2d-14            [-1, 25

In [9]:
from torch.utils.data import DataLoader
loader = DataLoader(dataset=dataset_train_AlexNet, shuffle=False, batch_size=1)

alexnet_features = []
y_train = []
alexnet.eval()
with torch.no_grad():
    for X_batch, y_batch in loader:
        alexnet_features.append(alexnet(X_batch))
        y_train.append(y_train)
print("Done feat extraction, total: {}".format(len(alexnet_features)))


Done feat extraction, total: 5020


In [9]:
knn = KNeighborsClassifier(n_neighbors=10).fit(alexnet_features, y_train)

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [ ]:
numpy_feat = np.zeros((len(alexnet_features), alexnet_features[0].shape[1]))
print((numpy_feat).shape)
i = 0
for i in range(0, len(alexnet_features)):
    numpy_feat[i]= alexnet_features[i].cpu().numpy()

(5020, 4096)
[ -2.88178611  -5.57203913  -1.57578468 ...  -8.63071632 -13.72015667
  -8.26701069]


In [14]:
def extractFeatures(img):
    transform = transforms.Compose([            
      transforms.Resize(256),                   
      transforms.CenterCrop(224),               
      transforms.ToTensor(),                    
      transforms.Normalize(                     
      mean=[0.485, 0.456, 0.406],               
      std=[0.229, 0.224, 0.225]                 
    )])
    
    img = img.convert('RGB')
    img_t = transform(img)
    batch_t = torch.unsqueeze(img_t, 0)
    out = alexnet(batch_t)
    assert out.shape == torch.Size([1, 4096])
    output = out.detach().numpy()
    output = np.reshape(output, 4096)
    return output

In [16]:
def prepare_data(path):
    X_train = []
    X_test = []
    y_train = []
    y_test = []
    i=1
    for folder_person in os.listdir(path):
        for folder_gesture in os.listdir(path + folder_person):
            for file in os.listdir(path + folder_person + "/" + folder_gesture)[0:30]:
                image = Image.open(os.path.join(path + folder_person + "/" + folder_gesture, file))
                if(i<5):
                    image_with_features = extractFeatures(image)
                    X_train.append(image_with_features)
                    y_train.append(int(folder_gesture[0:2]))
                    i+=1
                else:  
                    image_with_features = extractFeatures(image)
                    X_test.append(image_with_features)
                    y_test.append(int(folder_gesture[0:2]))
                    i=1
    return X_train, y_train, X_test, y_test

In [17]:
def model_building(model, X_train, X_test, y_train, y_test):
    model.fit(X_train,y_train)
    score = model.score(X_test,y_test)
    return score

In [20]:
DATASET_PATH = "../input/leapgestrecog/leapGestRecog/"
X_train, y_train, X_test, y_test = prepare_data(DATASET_PATH)

In [21]:
models = [(LinearSVC(), "Linear Support Vector"), 
          (RandomForestClassifier(), "Random forest"),
          (KNeighborsClassifier(n_neighbors=10), "KNN with 10 neighbors"),
          (GaussianNB(), "Naive Bayes")]

for model in models:    
    score = model_building(model[0], X_train, X_test, y_train, y_test)
    print("{} score: {}".format(model[1],score))

Linear Support Vector score: 0.9633333333333334
Random forest score: 0.9266666666666666
KNN with 10 neighbors score: 0.9833333333333333
Naive Bayes score: 0.805
